# Exploring different data files to see what they look like

In [8]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

### Exploring Atlas AI asset wealth file

In [13]:
# Open file for Rwanda boundary to crop

rwa_boundary = gpd.read_file("../data/locations/rwanda_boundary/RWA_adm0.shp")

In [17]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import box

# Open the TIFF file
tif_file = "../data/atlas_ai_files/Rwanda_Asset-Wealth-1912m_0_08_RWA_2020.tif"
boundary_file = "../data/locations/rwanda_boundary/RWA_adm0.shp"

with rasterio.open(tif_file) as src:
    # Read raster data and geometry
    array = src.read(1)
    transform = src.transform

    # Open the boundary shapefile
    boundary_gdf = gpd.read_file(boundary_file)

    # Make sure both GeoDataFrames have the same CRS
    boundary_gdf = boundary_gdf.to_crs(src.crs)

    # Create a bounding box that covers the extent of the raster
    xmin, ymin, xmax, ymax = src.bounds
    bbox = box(xmin, ymin, xmax, ymax)
    bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs=src.crs)

    # Intersect the bounding box with the boundary shapefile
    intersection = gpd.overlay(boundary_gdf, bbox_gdf, how='intersection')

    # Clip the raster to the intersection geometry
    clipped, transform = mask(src, shapes=intersection.geometry, crop=True)

    # Update the metadata for the clipped raster
    clipped_meta = src.meta.copy()
    clipped_meta.update({
        'height': clipped.shape[1],
        'width': clipped.shape[2],
        'transform': transform
    })

    # Save the clipped raster to a new TIFF file
    clipped_tif_file = 'file.tif'
    with rasterio.open(clipped_tif_file, 'w', **clipped_meta) as dst:
        dst.write(clipped)

# Print the clipped GeoDataFrame
print(intersection)


   ID_0  ISO NAME_ENGLI NAME_ISO NAME_FAO NAME_LOCAL  NAME_OBSOL  \
0   189  RWA     Rwanda   RWANDA   Rwanda     Rwanda         NaN   

                          NAME_VARIA  NAME_NONLA NAME_FRENC  ... CARICOM   EU  \
0  Belgian East Africa|Ruanda-Urundi         NaN     Rwanda  ...     0.0  0.0   

   CAN  ACP  Landlocked  AOSIS SIDS Islands  LDC  \
0  0.0  1.0         1.0    0.0  0.0     0.0  1.0   

                                            geometry  
0  POLYGON ((3389483.028 -116633.613, 3389504.685...  

[1 rows x 68 columns]


In [20]:
transform

Affine(1912.0, 0.0, 3214072.0,
       0.0, -1912.0, -116632.0)

In [27]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import box

# Open the TIFF file
tif_file = "../data/atlas_ai_files/Rwanda_Asset-Wealth-1912m_0_08_RWA_2020.tif"
boundary_file = "../data/locations/sector/Sector.shp"

with rasterio.open(tif_file) as src:
    # Read raster data and geometry
    array = src.read(1)
    transform = src.transform

    # Open the boundary shapefile
    boundary_gdf = gpd.read_file(boundary_file)

    # Make sure both GeoDataFrames have the same CRS
    boundary_gdf = boundary_gdf.to_crs(src.crs)

    # Create a bounding box that covers the extent of the raster
    xmin, ymin, xmax, ymax = src.bounds
    bbox = box(xmin, ymin, xmax, ymax)
    bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs=src.crs)

    # Intersect the bounding box with the boundary shapefile
    intersection = gpd.overlay(boundary_gdf, bbox_gdf, how='intersection')

    # Clip the raster to the intersection geometry
    clipped, transform = mask(src, shapes=intersection.geometry, crop=True)

    # Create a GeoDataFrame directly from the clipped raster
    shapes_gen = rasterio.features.shapes(clipped, transform=transform)
    features = [{'geometry': geometry, 'properties': {'pixel_value': value}}
                for (geometry, value) in shapes_gen]
    gdf_clipped = gpd.GeoDataFrame.from_features(features, crs=src.crs)

    gdf_clipped = gdf_clipped.to_crs(("EPSG:4326"))

# Print the GeoDataFrame with attribute table for the clipped TIFF file
print(gdf_clipped)

                                               geometry   pixel_value
0     POLYGON ((30.43550 -1.04766, 30.43550 -1.06484... -3.587138e-01
1     POLYGON ((30.45267 -1.04766, 30.45267 -1.06484... -3.595239e-01
2     POLYGON ((30.34962 -1.06484, 30.34962 -1.08201... -4.814316e-01
3     POLYGON ((30.36679 -1.06484, 30.36679 -1.08201... -4.526857e-01
4     POLYGON ((30.38397 -1.06484, 30.38397 -1.08201... -4.185151e-01
...                                                 ...           ...
6675  POLYGON ((29.37060 -2.81570, 29.37060 -2.83285... -5.126896e-01
6676  POLYGON ((29.42213 -2.79854, 29.45648 -2.79854...  3.402823e+38
6677  POLYGON ((29.50800 -2.81570, 29.50800 -2.83285... -1.502541e-01
6678  POLYGON ((29.52518 -2.81570, 29.52518 -2.83285... -1.446368e-01
6679  POLYGON ((30.86489 -2.11214, 30.88207 -2.11214...  3.402823e+38

[6680 rows x 2 columns]


In [28]:
gdf_clipped.head()

,geometry,pixel_value
0,"POLYGON ((30.43550 -1.04766, 30.43550 -1.06484...",-0.358714
1,"POLYGON ((30.45267 -1.04766, 30.45267 -1.06484...",-0.359524
2,"POLYGON ((30.34962 -1.06484, 30.34962 -1.08201...",-0.481432
3,"POLYGON ((30.36679 -1.06484, 30.36679 -1.08201...",-0.452686
4,"POLYGON ((30.38397 -1.06484, 30.38397 -1.08201...",-0.418515
